In [1]:
import pandas as pd
import numpy as np

from PIL import Image

df = pd.read_csv('all.csv', index_col=0)

# indexに1を足してデータセットと合わせている
df.index = df.index + 1

data = df[3078:4402]

In [2]:
# hen
uenohen = [4255, 4256, 4257, 4258, 4259, 4260, 4261]
shitanohen = [4165, 4166, 4167, 4168, 4169, 4170, 4171]
miginohen = [3829, 3828, 3827, 3826, 3825, 3824, 3823, 3822, 3821, 3820, 3819, 3818, 3817]
hidarinohen = [3780, 3779, 3778, 3777, 3776, 3775, 3774, 3773, 3772, 3771, 3770, 3769]

# kado
hidariue = [3781]
migiue = [3830]
hidarishita = [3768]
migishita = [3816]

In [3]:
def get_txt(data, direction_1, direction_2):
    n = len(data)
    for i in range(n-1):
        print(data[i], data[(i+1)], direction_1)
    for i in range(n):
        if i == 0:
            continue
        print(data[-i], data[-(i+1)], direction_2)

In [4]:
get_txt(hidarinohen, 3, 1)

3780 3779 3
3779 3778 3
3778 3777 3
3777 3776 3
3776 3775 3
3775 3774 3
3774 3773 3
3773 3772 3
3772 3771 3
3771 3770 3
3770 3769 3
3769 3770 1
3770 3771 1
3771 3772 1
3772 3773 1
3773 3774 1
3774 3775 1
3775 3776 1
3776 3777 1
3777 3778 1
3778 3779 1
3779 3780 1


In [5]:
# 写真をとる方角のグループに分ける
"""
group_1  1: 東, 2: 南, 3: 西, 4: 北
group_2  1: 南, 2: 西, 3: 北, 4: 東 
group_3  1: 西, 2: 北, 3: 東, 4: 南

  1
4   2
  3
group_1を基本とし他はそれに合わせる

"""

def north_img_num(dir):
    if dir < 50 or 300 < dir:
        return 4
    elif 50 < dir < 150:
        return 3
    else:
        return 2

df_direction = pd.DataFrame(index=data.index, columns=['north', 'west', 'south', 'east'])
df_direction['north'] = data['compass_direction'].map(north_img_num)
df_direction['west'] = df_direction['north'].map(lambda x: (x + 2) % 4 + 1)
df_direction['south'] = df_direction['north'].map(lambda x: (x + 1) % 4 + 1)
df_direction['east'] = df_direction['north'].map(lambda x: (x) % 4 + 1)
df_group_1_with_dir_num = data.join(df_direction)

group_1 = df_direction[df_direction['north']==4]
group_2 = df_direction[df_direction['north']==3]
group_3 = df_direction[df_direction['north']==2]

group_1 = list(group_1.index)
group_2 = list(group_2.index)
group_3 = list(group_3.index)

In [6]:
def check_group(a):
    for i in a:
        if i in group_3:
            print(f'{i} is group_3')
        elif i in group_2:
            print(f'{i} is group_2')
        else:
            print(f'{i} is group_1')

In [7]:
check_group(uenohen)

4255 is group_1
4256 is group_1
4257 is group_1
4258 is group_1
4259 is group_1
4260 is group_1
4261 is group_1


In [8]:
# get_txt(hidarinohen, 3, 1)

In [9]:
from ipywidgets import Textarea

def get_input(change):
    global Input
    Input = change["new"]

textarea = Textarea()
textarea.observe(get_input, names='value')
display(textarea)

Textarea(value='')

In [10]:
IN = iter(Input.split('\n')).__next__
def input():
    return IN()

N, M = map(int, input().split())

Graph = [[] for _ in range(4800)]

north = 4
south = 2
east = 1
west = 3

for i in range(M):
    a, b, direction = map(int, input().split())
    Graph[a].append((b, direction))

In [11]:
def node_int_to_string(node):
    node = str(node)
    temp = len(node)
    fill = 6 - temp
    fill_text = "0" * fill
    node = fill_text + node
    return node

def check_node_group(node):
    if node in group_3:
        return 3
    elif node in group_2:
        return 2
    else:
        return 1

def return_action(node_group, direction):
    north, south, east, west = 4, 2, 1, 3
    
    if node_group == 1:
        pass
    elif node_group == 2:
        north, south, east, west = east, west, south, north
    else:
        north, south, east, west = south, north, west, east
    
    if direction == 'north':
        action = north
        return action
    elif direction == 'south':
        action = south
        return action
    elif direction== 'east':
        action = east
        return action
    else:
        action = west
        return action

pictures = []
data_url = '../data/'

def return_image_and_moveable_node(image, direction):
    """
    Graph
    input image(node), action(direction)
    return image, next_node, and moveable_direction

    """
    currecnt_node = int(image[:6])
    node_group = check_node_group(currecnt_node)
    action = return_action(node_group, direction)
    
    moveable_nodes = Graph[currecnt_node]
    next_node = float('inf')
    for i in moveable_nodes:
        if i[1] == action:
            next_node = i[0]
    next_image = node_int_to_string(next_node) + "_" + str(action)
    _next_image = Image.open(data_url + next_image + ".jpg")
    pictures.append(_next_image)
    next_moveable_nodes = Graph[next_node]
    return next_image, next_node, next_moveable_nodes

In [12]:
return_image_and_moveable_node("004255", "south")

('004256_2', 4256, [(4257, 2), (4255, 4)])

In [13]:
start = '004255'
next_image, next_node, next_moveable_nodes = return_image_and_moveable_node(start, "south")

for i in range(5):
    next_image, next_node, next_moveable_nodes = return_image_and_moveable_node(next_image, "south")

In [16]:
pictures[0].save('anime.gif',save_all=True, append_images=pictures[1:],
optimize=False, duration=500, loop=0)